# 预处理数据

原来参考的代码，里面的scipy.misc版本太旧，所以这里我查了一下，相当于是放弃了scipy.misc库，改用别的库来弄：
https://blog.csdn.net/huang1024rui/article/details/119668502

In [7]:
import numpy as np
from scipy import misc
import os
import imageio
from skimage.transform import resize

dataset = []
examples = []
# images_background
data_root = "../../originalDataset/omniglot-master/python/"
alphabets = os.listdir(data_root + "images_background")
for alphabet in alphabets:
    characters = os.listdir(os.path.join(data_root, "images_background", alphabet))
    for character in characters:
        files = os.listdir(os.path.join(data_root, "images_background", alphabet, character))
        examples = []
        for img_file in files:
            img = resize(
                imageio.imread(os.path.join(data_root, "images_background", alphabet, character, img_file)), [28, 28]
            )
            examples.append(img) # example level append, making dim as 20X28X28
        dataset.append(examples) # character level append, making dim as (20*30)X20X28X28

# images_evaluation
data_root = "../../originalDataset/omniglot-master/python/"
alphabets = os.listdir(data_root + "images_evaluation")
for alphabet in alphabets:
    characters = os.listdir(os.path.join(data_root, "images_evaluation", alphabet))
    for character in characters:
        files = os.listdir(os.path.join(data_root, "images_evaluation", alphabet, character))
        examples = []
        for img_file in files:
            img = resize(
                imageio.imread(os.path.join(data_root, "images_evaluation", alphabet, character, img_file)), [28, 28]
            )
            examples.append(img)
        dataset.append(examples) # character level append, making dim as (n)X20X28X28

np.save(data_root + "dataset.npy", np.asarray(dataset))


# 跑一下

In [1]:
import numpy as np
import torch
import torch.nn as nn
import math
import numpy as np
import torch.nn.functional as F
from torch.autograd import Variable
import tqdm
import torch.backends.cudnn as cudnn
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt
%matplotlib inline 

In [2]:
batch_size = 16 # setting batch_size
classes_per_set = 20 # Number of classes per set
samples_per_class = 1 # as we are choosing it to be one shot learning, so we have 1 sample

In [3]:
x = np.load('../../originalDataset/omniglot-master/python/dataset.npy') # Load Data
print (x.shape)
print (type(x))
n_classes = x.shape[0] # total number of classes
x = np.reshape(x, newshape=(x.shape[0], x.shape[1], 28, 28, 1)) # expand dimension from (x.shape[0],x.shape[1],28,28)
np.random.shuffle(x) # shuffle dataset
x_train, x_val, x_test = x[:1200], x[1200:1411], x[1411:] # divide dataset in to train, val,ctest

print (x_train.shape)

(1623, 20, 28, 28)
<class 'numpy.ndarray'>
(1200, 20, 28, 28, 1)


In [4]:
def processes_batch(data, mu, sigma):
    return (data - mu) / sigma

# Normalize Dataset
x_train = processes_batch(x_train, np.mean(x_train), np.std(x_train))
x_val = processes_batch(x_val, np.mean(x_val), np.std(x_val))
x_test = processes_batch(x_test, np.mean(x_test), np.std(x_test))

# Defining dictionary of dataset
datatset = {"train": x_train, "val": x_val, "test": x_test}

In [5]:
def sample_batch(data):
    """
    Generates sample batch 
    :param : data - one of(train,test,val) our current dataset shape [total_classes,20,28,28,1]
    :return: [support_set_x,support_set_y,target_x,target_y] for Matching Networks

    每一个batch，都重复做batch_size次这样的操作：
    1. 从一千多个class里面选择20个，是support_set_x。每个类都只有1个图片，每个图片是28*28*1。
    2. 这20个class对应的support_set_y就是0～19.
    3. 从这20个class里面选择出1个作为target_x, target_y。
    嗯差不多就是这样。
    """
    support_set_x = np.zeros((batch_size, classes_per_set, samples_per_class, data.shape[2],
                              data.shape[3], data.shape[4]), np.float32)
    support_set_y = np.zeros((batch_size, classes_per_set, samples_per_class), np.int32)

    target_x = np.zeros((batch_size, data.shape[2], data.shape[3], data.shape[4]), np.float32)
    target_y = np.zeros((batch_size, 1), np.int32)
    for i in range(batch_size):
        choose_classes = np.random.choice(data.shape[0], size=classes_per_set, replace=False) # choosing random classes
        choose_label = np.random.choice(classes_per_set, size=1) # label set
        choose_samples = np.random.choice(data.shape[1], size=samples_per_class + 1, replace=False)
        x_temp = data[choose_classes] # choosing classes
        x_temp = x_temp[:, choose_samples] # choosing sample batch from classes chosen outputs (20X2X28X28X1)
        y_temp = np.arange(classes_per_set) # will return [0,1,2,3,...,19]
        support_set_x[i] = x_temp[:, :-1] ## (20X前面那1个X28*28*1)
        support_set_y[i] = np.expand_dims(y_temp[:], axis=1) # expand dimension ## shape will be (20,1)
        target_x[i] = x_temp[choose_label, -1] ## (1X后面那1个X28X28X1)
        target_y[i] = y_temp[choose_label] ## shape = (1,)
    return support_set_x, support_set_y, target_x, target_y # returns support of [batch_size, 20 classes per set, 1 sample, 28, 28,1]
    
def get_batch(dataset_name):
    """
    gen batch while training
    :param dataset_name: The name of dataset(one of "train","val","test")
    :return: a batch images

    实际上本质上还是sample_batch操作，只不过多了一步，就是把某一个形状是1的维度消去罢了。
    """
    support_set_x, support_set_y, target_x, target_y = sample_batch(datatset[dataset_name])
    support_set_x = support_set_x.reshape((support_set_x.shape[0], support_set_x.shape[1] * support_set_x.shape[2],
                                           support_set_x.shape[3], support_set_x.shape[4], support_set_x.shape[5]))
    support_set_y = support_set_y.reshape(support_set_y.shape[0], support_set_y.shape[1] * support_set_y.shape[2])
    return support_set_x, support_set_y, target_x, target_y

In [9]:
test_batch = get_batch("train")
for i in range(4):
    print(test_batch[i].shape)

(16, 20, 28, 28, 1)
(16, 20)
(16, 28, 28, 1)
(16, 1)


In [11]:
x_support_set, y_support_set, x_target, y_target = get_batch("train")

In [13]:
x_support_set = Variable(torch.from_numpy(x_support_set)).float()
y_support_set = Variable(torch.from_numpy(y_support_set), requires_grad=False).long()
x_target = Variable(torch.from_numpy(x_target)).float()
y_target = Variable(torch.from_numpy(y_target), requires_grad=False).squeeze().long()

In [18]:
y_target.shape ## 看来，y_target相比于原来的y_target，被squeeze了。

torch.Size([16])

In [19]:
y_support_set = y_support_set.unsqueeze(2)
y_support_set.shape

torch.Size([16, 20, 1])

In [20]:
sequence_length = y_support_set.size()[1] ## 20
batch_size = y_support_set.size()[0] ## 16
y_support_set_one_hot = Variable(
    torch.zeros(batch_size, sequence_length, 
                classes_per_set).scatter_(2,y_support_set.data,1), requires_grad=False)
y_support_set_one_hot.shape

torch.Size([16, 20, 20])

In [44]:
y_support_set_one_hot.shape

torch.Size([16, 20, 20])

In [53]:
y_support_set_one_hot

tensor([[[1., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 1., 0., 0.],
         [0., 0., 0.,  ..., 0., 1., 0.],
         [0., 0., 0.,  ..., 0., 0., 1.]],

        [[1., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 1., 0., 0.],
         [0., 0., 0.,  ..., 0., 1., 0.],
         [0., 0., 0.,  ..., 0., 0., 1.]],

        [[1., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 1., 0., 0.],
         [0., 0., 0.,  ..., 0., 1., 0.],
         [0., 0., 0.,  ..., 0., 0., 1.]],

        ...,

        [[1., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 1., 0., 0.],
         [0., 0., 0., 

In [21]:
batch_size=20
num_channels=1
lr=1e-3
image_size=28
classes_per_set=20
samples_per_class=1
keep_prob=0.0
fce=True
optim="adam"
wd=0

In [25]:
def convLayer(in_channels, out_channels, dropout_prob=0.0):
    """
    :param dataset_name: The name of dataset(one of "train","val","test")
    :return: a batch images
    """
    cnn_seq = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, 1, 1),
        nn.ReLU(True),
        nn.BatchNorm2d(out_channels),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Dropout(dropout_prob)
    )
    return cnn_seq

class Embeddings_extractor(nn.Module):
    def __init__(self, layer_size=64, num_channels=1, dropout_prob=0.5, image_size=28):
        super(Embeddings_extractor, self).__init__()
        """
        Build a CNN to produce embeddings
        :param layer_size:64(default)
        :param num_channels:
        :param keep_prob:
        :param image_size:
        """
        self.layer1 = convLayer(num_channels, layer_size, dropout_prob)
        self.layer2 = convLayer(layer_size, layer_size, dropout_prob)
        self.layer3 = convLayer(layer_size, layer_size, dropout_prob)
        self.layer4 = convLayer(layer_size, layer_size, dropout_prob)

        finalSize = int(math.floor(image_size / (2 * 2 * 2 * 2)))
        self.outSize = finalSize * finalSize * layer_size

    def forward(self, image_input):
        """
        :param: Image
        :return: embeddings
        """
        x = self.layer1(image_input)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = x.view(x.size()[0], -1)
        return x


In [26]:
test_ee = Embeddings_extractor()

In [29]:
x_support_set.shape

torch.Size([16, 20, 28, 28, 1])

In [33]:
x_support_set = x_support_set.permute(0, 1, 4, 2, 3) ## (16, 20, 1, 28, 28)
x_target = x_target.permute(0, 3, 1, 2) ## (16, 1, 28, 28, )

In [36]:
encoded_images = []
for i in np.arange(x_support_set.size(1)):
    input_img = x_support_set[:, i, :, :]
    gen_encode = test_ee(input_img)
    print(input_img.shape, gen_encode.shape)
    encoded_images.append(gen_encode)

torch.Size([16, 1, 28, 28]) torch.Size([16, 64])
torch.Size([16, 1, 28, 28]) torch.Size([16, 64])
torch.Size([16, 1, 28, 28]) torch.Size([16, 64])
torch.Size([16, 1, 28, 28]) torch.Size([16, 64])
torch.Size([16, 1, 28, 28]) torch.Size([16, 64])
torch.Size([16, 1, 28, 28]) torch.Size([16, 64])
torch.Size([16, 1, 28, 28]) torch.Size([16, 64])
torch.Size([16, 1, 28, 28]) torch.Size([16, 64])
torch.Size([16, 1, 28, 28]) torch.Size([16, 64])
torch.Size([16, 1, 28, 28]) torch.Size([16, 64])
torch.Size([16, 1, 28, 28]) torch.Size([16, 64])
torch.Size([16, 1, 28, 28]) torch.Size([16, 64])
torch.Size([16, 1, 28, 28]) torch.Size([16, 64])
torch.Size([16, 1, 28, 28]) torch.Size([16, 64])
torch.Size([16, 1, 28, 28]) torch.Size([16, 64])
torch.Size([16, 1, 28, 28]) torch.Size([16, 64])
torch.Size([16, 1, 28, 28]) torch.Size([16, 64])
torch.Size([16, 1, 28, 28]) torch.Size([16, 64])
torch.Size([16, 1, 28, 28]) torch.Size([16, 64])
torch.Size([16, 1, 28, 28]) torch.Size([16, 64])


In [38]:
# produce embeddings for target images
gen_encode = test_ee(x_target)## (16, 64)
encoded_images.append(gen_encode) ## 维度变成了(21, 16, 64)
output = torch.stack(encoded_images,dim=0) ## 把encoded_images合并成一个向量，形状(21, 16, 64)
output.shape

torch.Size([21, 16, 64])

In [41]:
class DistanceNetwork(nn.Module):
    """
    This model calculates the cosine distance between each of the support set embeddings and 
    the target image embeddings.
    """

    def __init__(self):
        super(DistanceNetwork, self).__init__()

    def forward(self, support_set, input_image):
        """
        forward pass
        :param support_set:the embeddings of the support set images.shape[sequence_length,batch_size,64]
        :param input_image: the embedding of the target image,shape[batch_size,64]
        :return:shape[batch_size,sequence_length]
        """
        ## 输入的形状，我们分别设定为 (20, 16, 64) (16, 64)
        eps = 1e-10
        similarities = []
        for support_image in support_set: ## support_image的形状(16,64)
            sum_support = torch.sum(torch.pow(support_image, 2), 1)
            print(sum_support.shape) ## (16,1)
            support_manitude = sum_support.clamp(eps, float("inf")).rsqrt()
            dot_product = input_image.unsqueeze(1).bmm(support_image.unsqueeze(2)).squeeze()
            cosine_similarity = dot_product * support_manitude
            similarities.append(cosine_similarity)
        similarities = torch.stack(similarities)
        return similarities.t()

In [42]:
self_c = DistanceNetwork()
similarites = self_c(
    support_set=output[:-1], ## shape是(20,16,64)
    input_image=output[-1] ## (16,64)
)

torch.Size([16])
torch.Size([16])
torch.Size([16])
torch.Size([16])
torch.Size([16])
torch.Size([16])
torch.Size([16])
torch.Size([16])
torch.Size([16])
torch.Size([16])
torch.Size([16])
torch.Size([16])
torch.Size([16])
torch.Size([16])
torch.Size([16])
torch.Size([16])
torch.Size([16])
torch.Size([16])
torch.Size([16])
torch.Size([16])


In [48]:
values, indices = similarites.max(axis=1)

In [52]:
indices

tensor([ 2, 10, 18,  3, 10, 19,  5,  3, 18, 12,  8, 15,  9, 15,  2, 13])

In [51]:
indices.squeeze()

tensor([ 2, 10, 18,  3, 10, 19,  5,  3, 18, 12,  8, 15,  9, 15,  2, 13])